In [165]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('../input/ghouls-goblins-and-ghosts-boo/train.csv') # обучающая выборка
X_test = pd.read_csv('../input/ghouls-goblins-and-ghosts-boo/test.csv') # тестовая выборка
print(dataset.shape)
print(dataset[:10])

(371, 7)
   id  bone_length  rotting_flesh  hair_length  has_soul  color    type
0   0     0.354512       0.350839     0.465761  0.781142  clear   Ghoul
1   1     0.575560       0.425868     0.531401  0.439899  green  Goblin
2   2     0.467875       0.354330     0.811616  0.791225  black   Ghoul
3   4     0.776652       0.508723     0.636766  0.884464  black   Ghoul
4   5     0.566117       0.875862     0.418594  0.636438  green   Ghost
5   7     0.405680       0.253277     0.441420  0.280324  green  Goblin
6   8     0.399331       0.568952     0.618391  0.467901  white  Goblin
7  11     0.516224       0.536429     0.612776  0.468048  clear   Ghoul
8  12     0.314295       0.671280     0.417267  0.227548   blue   Ghost
9  19     0.280942       0.701457     0.179633  0.141183  white   Ghost


In [166]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 7 columns):
id               371 non-null int64
bone_length      371 non-null float64
rotting_flesh    371 non-null float64
hair_length      371 non-null float64
has_soul         371 non-null float64
color            371 non-null object
type             371 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 20.4+ KB


In [167]:
dataset.describe()

,id,bone_length,rotting_flesh,hair_length,has_soul
count,371.000000,371.000000,371.000000,371.000000,371.000000
mean,443.676550,0.434160,0.506848,0.529114,0.471392
std,263.222489,0.132833,0.146358,0.169902,0.176129
min,0.000000,0.061032,0.095687,0.134600,0.009402
25%,205.500000,0.340006,0.414812,0.407428,0.348002
50%,458.000000,0.434891,0.501552,0.538642,0.466372
75%,678.500000,0.517223,0.603977,0.647244,0.600610
max,897.000000,0.817001,0.932466,1.000000,0.935721


In [168]:
dataset.describe(include=['O'])

,color,type
count,371,371
unique,6,3
top,white,Ghoul
freq,137,129


In [169]:
X_train = dataset.iloc[:, 1:6].values
Y_train = dataset.iloc[:, 6].values
X_test_id = X_test['id']
X_test = X_test.iloc[:, 1:6].values

In [170]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
X_train[:, 4] = labelencoder_X_1.fit_transform(X_train[:, 4])

labelencoder_X_2 = LabelEncoder()
X_test[:, 4] = labelencoder_X_2.fit_transform(X_test[:, 4])

labelencoder_Y_2 = LabelEncoder()
temp = Y_train
Y_train = labelencoder_Y_2.fit_transform(Y_train)


In [171]:
one_hot_encoder = OneHotEncoder(categorical_features = [4])
X_train = one_hot_encoder.fit_transform(X_train).toarray()
X_test = one_hot_encoder.fit_transform(X_test).toarray()
print(X_train.shape)
print(X_test.shape)

(371, 10)
(529, 10)


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data w

In [172]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.2)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(296, 10) (296,) (75, 10) (75,)


In [173]:
# импортируем нужные библиотеки
import torch
import numpy as np # всегда пригодится :)

# инициализируем девайс
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# добавляем типовую функцию "шаг обучения"
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

In [227]:
from torch import optim, nn

model = torch.nn.Sequential(
    nn.Linear(10, 270),
    nn.ReLU(),
    nn.Linear(270, 3))
lr = 0.001
n_epochs = 5000
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay = 0.001)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum = *, weight_decay = *)
#optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay = *)

train_step = make_train_step(model, loss_fn, optimizer)

from sklearn.utils import shuffle

for epoch in range(n_epochs):
    x_train, y_train = shuffle(x_train, y_train) # случайно перетасуем данные
    X = torch.FloatTensor(x_train)
    y = torch.LongTensor(y_train)
    loss = train_step(X, y)
print(loss)

0.33751705288887024


In [228]:
# оцениваем модель:
test_var = torch.FloatTensor(x_val)
with torch.no_grad():
    result = model(test_var)
values, labels = torch.max(result, 1)
num_right = np.sum(labels.data.numpy() == y_val)
print('Точность {:.2f}'.format(num_right / len(y_val)))

Точность 0.75


In [229]:
import csv

with torch.no_grad():
    result = model(torch.FloatTensor(X_test))
values, labels = torch.max(result, 1)
type_ = labels.data.numpy()
submission = [['id', 'type']]
for i in range(len(type_)):
    ans = ''
    if type_[i] == 0:
        ans = temp[4]
    if type_[i] == 1:
        ans = temp[0]
    if type_[i] == 2:
        ans = temp[1]
    submission.append([X_test_id[i], ans])

In [230]:
with open('submission.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerows(submission)
    
print('Файл для соревнования подготовлен!')

Файл для соревнования подготовлен!


In [231]:
!tail submission.csv

In [ ]:
%env KAGGLE_USERNAME=здесь-ваш-ник
%env KAGGLE_KEY=здесь-ваш-ключ
!export -p | grep KAGGLE_

In [ ]:
!pip install kaggle --upgrade

In [ ]:
!kaggle competitions submit -c ghouls-goblins-and-ghosts-boo -f submission.csv -m "submission"